In [5]:
%reload_kedro

[05/25/24 12:37:35] INFO     Resolved project path as:                                              ]8;id=321514;file:///home/ted/Desktop/DEV/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=889991;file:///home/ted/Desktop/DEV/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             /home/ted/Desktop/DEV/marketing-analytics-dashboard/kedro-project.                    
                             To set a different path, run '%reload_kedro <project_root>'                           

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ted/Desktop/DEV/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/kedro/io/c │
│ ore.py:152 in from_config                                                                        │
│                                                                                                  │
│   149 │   │                                                                                      │
│   150 │   │   """                                                                                │
│   151 │   │   try:                                                                               │
│ ❱ 152 │   │   │   class_obj, config = parse_dataset_definition(                                  │
│   153 │   │   │   │   config, load_version, save_version                                         │
│   154 │   │   │   )                                                                              │
│   155 │   │   except Exception as exc:                                                           │
│                                                                                                  │
│ /home/ted/Desktop/DEV/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/kedro/io/c │
│ ore.py:405 in parse_dataset_definition                                                           │
│                                                                                                  │
│   402 │   │   │   │   class_obj = tmp                                                            │
│   403 │   │   │   │   break                                                                      │
│   404 │   │   else:                                                                              │
│ ❱ 405 │   │   │   raise DatasetError(f"Class '{dataset_type}' not found, is this a typo?")       │
│   406 │                                                                                          │
│   407 │   if not class_obj:                                                                      │
│   408 │   │   class_obj = dataset_type                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
DatasetError: Class 'pandas.CSVDataSet' not found, is this a typo?

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_ipython().run_line_magic('reload_kedro', '')                                             │
│   2                                                                                              │
│                                                                                                  │
│ /home/ted/Desktop/DEV/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/IPython/co │
│ re/interactiveshell.py:2480 in run_line_magic                                                    │
│                                                                                                  │
│   2477 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2478 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2479 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2480 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2481 │   │   │                                                                                 │
│   2482 │   │   │   # The code below prevents the output from being displayed                     │
│   2483 │   │   │   # w

24/05/25 20:05:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3333062 ms exceeds timeout 120000 ms
24/05/25 20:05:36 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/25 20:05:36 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 